# Librerías

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import  pywhatkit as kit
from datetime import datetime
from time import sleep
import os
import time

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
service = Service('/usr/bin/chromedriver')

Comedor Universitario Siberia
SELECCIONAR
Comedor ESC AGROTECNICA
SELECCIONAR
Comedor Universitario Área Salud
SELECCIONAR
Comedor Universitario FCEIA
SELECCIONAR
Comedor Universitario Zavalla
SELECCIONAR
Comedor Universitario Casilda
SELECCIONAR
Comedor Universitario Área Centro

In [31]:
cod_comedores = {
    'siberia' : 0,
    'agrotecnia' : 1,
    'salud' : 2,
    'fceia' : 3,
    'zaballa' : 4,
    'casilda' : 5,
    'centro' : 6,
}

In [39]:
cod_menues_salud = {
    'des' : 0,
    'alm_llevar_14' : 1,
    'alm_llevar_13' : 2,
    'alm_llevar_12' : 3,
    'alm_llevar_11' : 4,
    'alm_no_llevar_14' : 5,
    'alm_no_llevar_13' : 6,
    'alm_no_llevar_12' : 7,
    'mer' : 8,
    'cen_llevar' : 9,
    'cen_no_llevar_20' : 10,
    'cen_no_llevar_19.30' : 11,
}

In [55]:
# Abrir la página web
driver = webdriver.Chrome(options=chrome_options)
try:
    url_login = "https://comedores.unr.edu.ar/"
    url_reservas = "https://comedores.unr.edu.ar/comedor-reserva/reservar"
    driver.get(url_login)
    # Capturar el elemento formulario
    e_form = WebDriverWait(driver=driver, timeout=2).until(
        EC.presence_of_element_located(locator=(By.ID, "form-login"))
    )
    print('formulario recibido')
    # Capturar los elementos del formulario
    e_input_dni = e_form.find_element(by=By.NAME,value='form-login[dni]')
    e_input_clave = e_form.find_element(by=By.NAME,value='form-login[clave]')
    e_boton = e_form.find_element(by=By.NAME,value='botones[botonEnviar]')
    # Completar el forumalario y enviarlo
    e_input_dni.send_keys('34178096')
    e_input_clave.send_keys('facundo0301')
    e_boton.click()
    print('formulario enviado')
    # Ingresar a la sección de reserva
    driver.get(url_reservas)
    e_contenedor_comedores = WebDriverWait(driver=driver, timeout=2).until(
        EC.presence_of_element_located(locator=(By.CLASS_NAME,'reservar-comedores-contenedor'))
    )
    titulo = driver.find_element(by=By.TAG_NAME,value='h1')
    e_botones_comedores = e_contenedor_comedores.find_elements(by=By.TAG_NAME,value='button')
    n_boton_salud = cod_comedores['salud']
    e_boton_salud = e_botones_comedores[n_boton_salud]
    # Ingresar al comedor de Salud
    e_boton_salud.click()
    print('Ingreso a Salud')
    titulo = driver.find_element(by=By.TAG_NAME,value='h1')
    print(titulo.text)
    op_reservas = driver.find_elements(by=By.CLASS_NAME,value='reservar-servicio')
    n_op_alm_llevar_11 = cod_menues_salud['alm_llevar_11']
    op_alm_llevar_11 = op_reservas[n_op_alm_llevar_11]
    op_alm_llevar_11.click()
    driver.implicitly_wait(20)
    print('avancemos en reservar....')
    e_resumen = driver.find_element(by=By.CLASS_NAME,value='reserva-resumen-card')
    print(e_resumen.text)
            
except Exception as e:
    print(f"Ocurrió un error al visitar la página: {e}.")

finally:
    # Close the WebDriver
    print(driver.title)
    print(driver.current_url)   
    driver.quit()

formulario recibido
formulario enviado
Ingreso a Salud
Reserva de turno para Comedor Universitario Área Salud
avancemos en reservar....
Información
Comedor Universitario Área Salud
ALMUERZO PARA LLEVAR
De 11:00 a 12:00
Reserva de turno
https://comedores.unr.edu.ar/comedor-reserva/reservar
